In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# To ADD:

# Normalization?
# Surrogate


In [ ]:
vaccinations = pd.read_excel("/content/Vaccinations_bycountybyday.xlsx")

In [ ]:
from sklearn.preprocessing import StandardScaler

infections = pd.read_csv("/content/ConfirmedCasesUSACounties2020-23.csv", sep=";")

infections = infections[infections['State'] == "GA"]
infections = infections.reset_index(drop=True)

dict_id = {}
for index, row in infections.iterrows():
    dict_id[row['countyFIPS']] = index

infections = infections.loc[:, (infections != 0).any(axis=0)]
infections

,countyFIPS,County Name,State,StateFIPS,03/03/20,04/03/20,05/03/20,06/03/20,07/03/20,08/03/20,...,14/07/23,15/07/23,16/07/23,17/07/23,18/07/23,19/07/23,20/07/23,21/07/23,22/07/23,23/07/23
0,0,Statewide Unallocated,GA,13,0,0,0,0,0,0,...,86623,86623,86623,86623,86623,86623,86623,86623,86623,86623
1,13001,Appling County,GA,13,0,0,0,0,0,0,...,3846,3846,3846,3846,3846,3846,3846,3846,3846,3846
2,13003,Atkinson County,GA,13,0,0,0,0,0,0,...,1703,1703,1703,1703,1703,1703,1703,1703,1703,1703
3,13005,Bacon County,GA,13,0,0,0,0,0,0,...,2853,2853,2853,2853,2853,2853,2853,2853,2853,2853
4,13007,Baker County,GA,13,0,0,0,0,0,0,...,343,343,343,343,343,343,343,343,343,343
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,13313,Whitfield County,GA,13,0,0,0,0,0,0,...,30450,30450,30450,30450,30450,30450,30450,30450,30450,30450
156,13315,Wilcox County,GA,13,0,0,0,0,0,0,...,902,902,902,902,902,902,902,902,902,902
157,13317,Wilkes County,GA,13,0,0,0,0,0,0,...,1195,1195,1195,1195,1195,1195,1195,1195,1195,1195
158,13319,Wilkinson County,GA,13,0,0,0,0,0,0,...,1887,1887,1887,1887,1887,1887,1887,1887,1887,1887


In [ ]:
connectivity = pd.read_csv("/content/county_county.tsv", sep='\t')
connectivity = connectivity[
    connectivity['fr_loc'].isin(infections['countyFIPS']) &
    connectivity['user_loc'].isin(infections['countyFIPS'])]

connectivity['node1-id'] = connectivity['user_loc'].replace(dict_id).astype(int)
connectivity['node2-id'] = connectivity['fr_loc'].replace(dict_id).astype(int)

connectivity.drop(columns=['fr_loc', 'user_loc'], inplace=True)

average_scaled_sci = connectivity['scaled_sci'].mean()
new_observations = pd.DataFrame({
    'node1-id': np.zeros(159, dtype='int'),
    'node2-id': range(159),
    'scaled_sci': average_scaled_sci
})
connectivity = pd.concat([connectivity, new_observations], ignore_index=True)
connectivity.sort_values(by=['node1-id', 'node2-id'], inplace=True, ascending=True)


scaled_sci_column = connectivity['scaled_sci'].values.reshape(-1, 1)
scaler = StandardScaler()
connectivity['scaled_sci'] = scaler.fit_transform(scaled_sci_column)
connectivity

<ipython-input-87-8e727493cb3e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  connectivity['node1-id'] = connectivity['user_loc'].replace(dict_id).astype(int)
<ipython-input-87-8e727493cb3e>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  connectivity['node2-id'] = connectivity['fr_loc'].replace(dict_id).astype(int)
<ipython-input-87-8e727493cb3e>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org

,scaled_sci,node1-id,node2-id
25281,0.000000,0,0
25282,0.000000,0,1
25283,0.000000,0,2
25284,0.000000,0,3
25285,0.000000,0,4
...,...,...,...
25276,-0.116782,159,155
25277,0.123443,159,156
25278,-0.110147,159,157
25279,-0.110257,159,158


In [ ]:
edges_index = torch.tensor(connectivity[['node1-id', 'node2-id']].values, dtype=torch.long).t()
edges_feat = torch.tensor(connectivity['scaled_sci'].values, dtype=torch.float64).t()

from torch.utils.data import DataLoader, TensorDataset

# Extract the time series data from the DataFrame
time_series_data = infections.iloc[:, 3:].values  # Assuming columns from the forth correspond to your single feature

# Define the window size (number of time steps to consider for each prediction)
window_size = 3

# Create input-output pairs with a sliding window
input_data, output_data = [], []
for i in range(len(time_series_data) - window_size):
    input_slice = time_series_data[:, i:i+window_size]  # Use columns from i to i+window_size
    input_data.append(input_slice)

# Convert the lists to NumPy arrays
input_data = np.array(input_data)

# Convert NumPy arrays to PyTorch tensors
input_tensor = torch.tensor(input_data, dtype=torch.float32)

input_tensor

tensor([[[1.3000e+01, 0.0000e+00, 0.0000e+00],
         [1.3000e+01, 0.0000e+00, 0.0000e+00],
         [1.3000e+01, 0.0000e+00, 0.0000e+00],
         ...,
         [1.3000e+01, 0.0000e+00, 0.0000e+00],
         [1.3000e+01, 0.0000e+00, 0.0000e+00],
         [1.3000e+01, 0.0000e+00, 0.0000e+00]],

        [[0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00],
         ...,
         [0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00]],

        [[0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00],
         ...,
         [0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00]],

        ...,

        [[1.7863e+04, 1.7863e+04, 1.7863e+04],
         [6.0100e+02, 6.0100e+02, 6.1500e+02]

#Graph Neural Network with PyTorch Geometric

In [1]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.2 MB/s eta 0:00:00


In [2]:
import torch
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from torch_geometric.data import Data

In [ ]:
graph_data = Data(x=input_data, edge_index=edges_index, edge_attr=edges_feat)

graph_data

Data(x=[157, 160, 3], edge_index=[2, 25440], edge_attr=[25440])

In [ ]:
# Idea:
# (1) Same as normal PyTorch module but use Layers of PyTorch Geometric (choose which ones)
# (2) Single node prediction given feats and graph----> one prediction for each node

class GCN(torch.nn.Module):
    def __init__(self):
        # Init parent
        super(GCN, self).__init__()
        torch.manual_seed(42)

        # Layers --- How many message passing layers? 2-3-4! Do we want to augment the embedding? Yes! Which Message Passing? GCN, SGN, AGN
        embedding_size = 32
        self.mex1 = GCNConv(graph_data.num_features, embedding_size)
        self.mex2 = GCNConv(embedding_size, embedding_size)
        self.mex3 = GCNConv(embedding_size, embedding_size)

        # Output layer
        self.out = Linear(embedding_size, 1) # Final MLP

    def forward(self, x, edge_index, batch_index):

        hidden = self.mex1(x, edge_index)
        hidden = F.relu(hidden)
        hidden = self.mex2(hidden, edge_index)
        hidden = F.relu(hidden)
        hidden = self.mex3(hidden, edge_index)
        hidden = F.relu(hidden)
        out = self.out(hidden)

        return out, hidden

model = GCN()
print(model)
print("Number of parameters: ", sum(p.numel() for p in model.parameters()))

GCN(
  (mex1): GCNConv(3, 32)
  (mex2): GCNConv(32, 32)
  (mex3): GCNConv(32, 32)
  (out): Linear(in_features=32, out_features=1, bias=True)
)
Number of parameters:  2273


In [ ]:
# Surrogate class implementation - check what is needed... I don't think Recovered are needed. Adjust with sum equal to 1

class SIRVSurrogate:
  def __init__(self, vacc, recover, infect, pop): # time series data
    self.V = vacc
    self.alpha = vacc_rate # compute
    self.R = recover
    self.beta = recover_rate # compute
    self.I = infect

  def forward(self, inf_rate):

    S = 1 - self.I - self.R - self.V
    dIdt = inf_rate * S * self.I - self.beta * self.I
    new_infected = dIdt + self.I
    # takes infection rate, outputs the number of infected

    return new_infected
  def backward(dLdinfected):
    # dLoss/doutput ---> dLoss/doutput * doutput/dinput
    dnewIdrate = (1 - self.I - self.R - self.V) * self.I
    return dnewIdrate * dLdinfected


In [4]:
#LSTM: for ablation study, we want it to be able to have input from either the SIRV surrogate, or the GNN, or just the data that goes into those
LSTMInputSize = 0
includeGNN = True
includeSIRV = True
numCounties = 159#in Georgia

if includeSIRV:
  #want to take in output of SIRV, along with other tabular data (we will add this in at a later point)
  LSTMInputSize = numCounties
elif includeGNN:
  #include output of GNN
  LSTMInputSize = numCounties
else:
  #use input to the GNN, since there is no output from SIRV or GNN to use
  LSTMInputSize = 3
#future work: hyperparameter tuning for hidden layer size
LSTM = torch.nn.LSTM(LSTMInputSize, hidden_size = 20)